#Install dependencies

In [ ]:
!pip install -q torch transformers sentence-transformers faiss-cpu flask pyngrok pyyaml bitsandbytes

# Import libraries and setup

In [ ]:
import os
import json
import threading
import time
import requests
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok


# Load the knowledge base data

In [ ]:
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "Shoplite User Registration Process",
        "content": """To create a Shoplite account, users begin by navigating to the registration page and entering their email address, password, and basic profile details such as full name and shipping address. Passwords must meet security requirements: at least eight characters long, with one number, and one special character. Once the form is submitted, Shoplite automatically sends a verification email that must be completed within 24 hours. Until verification is complete, accounts remain restricted from placing orders, following sellers, or saving items. This ensures that all accounts are tied to valid, unique email addresses.

Shoplite offers two account types: buyer accounts and seller accounts. Buyer accounts are free and grant access to browsing, ordering, reviewing products, and maintaining wishlists. Seller accounts, on the other hand, require additional business verification. Applicants must provide valid business registration documents, proof of identity, tax information, and bank account details. Seller applications are reviewed within two to three business days, during which Shoplite ensures compliance with regulations and fraud prevention policies.

Duplicate email registrations are not allowed. If a user attempt is made to register with a duplicate email, the user is prompted to reset their password instead of creating a new account. Customer support agents are trained to assist with registration issues, resend verification emails, and resolve duplicate account conflicts.

Shoplite also monitors for suspcious activity, such as multiple account registrations from the same IP or device. In such cases, additional verification may be required. Once registration is verified, users can log in, manage their accounts, and begin using all of Shoplite’s features."""
    },
    {
        "id": "doc2",
        "title": "Shoplite Shopping Cart Features",
        "content": """The Shoplite shopping cart is designed to make the shopping experience smooth and flexible for buyers. Users can add products to the cart from any product page and continue browsing without interruption. The cart supports mixed-seller orders, meaning items from different merchants can be purchased together. Shipping fees for each seller are calculated individually to maintain accuracy. Logged-in users benefit from persistent cart storage across sessions, ensuring that their items remain saved even after logging out or switching devices. For users not yet ready to buy, items can be marked as “Save for Later.”

During checkout, the cart provides a real-time calculation of taxes and shipping fees. Customers can apply one promotional code per order, with the system automatically validating the code before applying it. Invalid or expired codes are rejected with clear feedback to the user. Estimated delivery times for each item are shown based on availability and courier options. The cart also automatically flags items that are running low in stock, encouraging customers to complete purchases before items sell out.

To ensure fairness, stock levels are verified again during checkout. If an item sells out while a user is completing payment, the system will immediately notify them and remove the unavailable item. Abandoned cart reminders are optionally sent by email to encourage users to finish transactions. Changes made on the mobile app are instantly synced to the web cart, providing a unified multi-device experience. These features collectively enhance user convenience while preventing overselling and ensuring transparent pricing."""
    },
    {
        "id": "doc3",
        "title": "Shoplite Payment Methods and Security",
        "content": """Shoplite supports multiple payment methods to accommodate user preferences. Customers can pay using Visa, Mastercard, and major debit cards. In certain regions, Cash on Delivery (COD) is also available. COD allows users to pay at the time of delivery, but refunds for COD orders are always issued via bank transfer. Refunds typically take 7–10 business days depending on bank processing times. Shoplite does not yet support digital wallets such as Apple Pay or PayPal, though these may be added in future updates.

All online transactions are encrypted and processed through PCI-compliant payment gateways. Shoplite does not store full card details on its servers, ensuring that sensitive information remains protected. Payment receipts are emailed immediately after a transaction is confirmed, and customers can download invoices from their order history page. For additional security, Shoplite provides fraud detection that monitors unusual payment patterns such as repeated failed attempts or mismatched billing addresses. Orders flagged as suspicious may be temporarily placed on hold until verified by customer support.

Users can manage their payment preferences from the account dashboard, including updating billing addresses and removing outdated payment methods. If a payment fails, customers receive guidance on retrying or switching to an alternative method. By combining multiple payment options, strict compliance with security standards, and robust fraud detection, Shoplite ensures a safe, reliable checkout experience for all customers."""
    },
    {
    "id": "doc4",
    "title": "Shoplite Order Tracking and Delivery",
    "content": """After placing an order on Shoplite, customers can track their purchase step by step. Orders pass through several distinct stages: Placed, Packed, Shipped, Out for Delivery, and Delivered. Each stage automatically triggers email and push notifications so that customers are always informed of their order’s status. Within the “My Orders” section of the website and mobile app, customers can view real-time updates, expected delivery dates, and courier details. Tracking numbers are generated once an order is shipped, and users can follow their package directly through integrated courier systems.

Delivery times depend on the service chosen. Standard delivery typically takes 3–5 business days, while express delivery, available for an extra fee, promises 1–2 business days. During checkout, estimated delivery windows are calculated using stock availability, warehouse location, and courier capacity. Shoplite ensures that if an order is delayed more than seven days beyond its estimated delivery, the customer can request a refund for shipping charges.

If a delivery attempt fails, couriers make up to two additional attempts before returning the package to the seller. In such cases, Shoplite processes a refund once the item is returned and inspected. For high-value orders, a delivery confirmation signature or app-based acknowledgment may be required to confirm receipt. Currently, Shoplite offers domestic deliveries only and does not yet support international shipping. This structured approach provides transparency, reliability, and flexibility for customers expecting timely deliveries."""
    },
    {
        "id": "doc5",
        "title": "Shoplite Return and Refund Policies",
        "content": """Shoplite’s return and refund policy aims to balance customer satisfaction with seller protection. Customers may request a return within 14 days of receiving an item, provided the product is unused and in original packaging. Certain categories are non-returnable, including consumables, hygiene-related items, and products marked as “Final Sale.” To start a return, customers must request authorization through their account. Once approved, returns are picked up by courier and sent back to the seller.

Refunds are processed within 7–10 business days after the item is inspected and verified. For credit card payments, refunds are sent back to the original payment method. For Cash on Delivery orders, customers must provide bank account details to receive refunds via transfer. Store credit is also an option in some cases, allowing customers to use funds for future purchases. If an item arrives defective or damaged, Shoplite covers return shipping costs. Otherwise, customers may be responsible for return shipping fees.

The refund system is tightly integrated with Shoplite’s fraud detection. Abusive patterns, such as frequent returns or misuse of the policy, can trigger account review. Shoplite also requires photo evidence for damaged goods before approving returns. This ensures fairness and minimizes disputes. Overall, the policy provides a flexible and trustworthy process that protects customers while ensuring sellers are not unfairly impacted."""
    },
    {
        "id": "doc6",
        "title": "Shoplite Product Reviews and Ratings",
        "content": """Shoplite enables customers to share feedback through a structured reviews and ratings system. Only verified buyers—those who have successfully purchased and received the product—can leave reviews. This ensures authenticity and prevents fraudulent or biased reviews from affecting product credibility. Customers can leave star ratings ranging from one to five, write descriptive text, and upload photos of the item they received. Reviews help inform other shoppers while also providing valuable feedback for sellers.

All reviews undergo automated moderation before being published. Shoplite’s system checks for inappropriate language, spam, and irrelevant content. Reviews that pass the filter are posted publicly, while questionable entries are flagged for human moderation. Sellers cannot delete or modify customer reviews; however, they may respond publicly to address issues, clarify policies, or thank customers for their feedback. This system encourages transparency and accountability.

The review section displays an overall rating average as well as breakdowns by star level. Customers can sort reviews by most recent, most helpful, or ratings. To prevent abuse, Shoplite monitors suspicious patterns such as multiple reviews from the same account or IP address. Reviews found to violate policy can be removed by the moderation team. In addition, reviews with attached images are given more visibility to enhance credibility. By ensuring authenticity and fair moderation, Shoplite’s review system builds trust and helps customers make confident purchase decisions."""
    },
    {
        "id": "doc7",
        "title": "Shoplite Seller Account Setup and Management",
        "content": """To create a seller account through the Seller Registration page. Applicants are required to submit valid business documents, proof of registration, tax identification numbers, and banking information. Identity verification is also part of the process. Shoplite’s compliance team typically reviews applications within two to three business days. Once approved, sellers gain access to the Seller Dashboard, where they can manage inventory, orders, and financial reporting.

The Seller Dashboard provides detailed sales analytics, including revenue breakdowns, customer demographics, and product performance. Sellers can also manage shipping preferences, set product pricing, and run promotions directly through this interface. Shoplite offers training materials and guides to help new sellers understand best practices for maximizing sales and maintaining high customer satisfaction.

To maintain marketplace integrity, Shoplite enforces policies regarding prohibited items, intellectual property, and customer service standards. Sellers found violating these rules may face account suspension or termination. Customer reviews and ratings are tied to seller performance, and consistent negative feedback may trigger account review. Sellers are also required to maintain fulfillment timelines to avoid late shipment penalties. Shoplite periodically audits seller activity to ensure fair practices and regulatory compliance. Overall, the seller management system provides both flexibility and accountability for merchants looking to scale their businesses on Shoplite."""
    },
    {
        "id": "doc8",
        "title": "Shoplite Inventory Management for Sellers",
        "content": """Inventory management is critical for maintaining a smooth buyer experience on Shoplite. Sellers manage stock levels directly through the Seller Dashboard or via API integration. The system provides real-time stock validation, ensuring that buyers cannot purchase items once they are sold out. Shoplite enforces conditional stock decrements during checkout to prevent overselling: stock is temporarily reserved at the time of payment initiation and deducted only upon successful payment confirmation.

Sellers receive automated notifications when inventory falls below customizable thresholds, helping them restock in time. Backorders are generally not allowed, although pre-orders for upcoming products can be configured with special approval. Shoplite also provides detailed reports that track sales velocity and inventory turnover, helping sellers forecast demand and plan restocks efficiently.

The platform automatically updates item status to “Sold Out” when stock reaches zero, removing it from search and browse results. This prevents customer frustration caused by attempting to purchase unavailable products. Additionally, Shoplite integrates with courier systems to ensure that shipping labels cannot be generated for out-of-stock items. Sellers using API integration can synchronize their external ERP or POS systems with Shoplite to maintain accurate inventory across multiple channels. By combining real-time validation, analytics, and automation, Shoplite ensures a reliable, oversell-free shopping experience for customers."""
    },
    {
        "id": "doc9",
        "title": "Shoplite Commission and Fee Structure",
        "content": """Shoplite uses a transparent commission and fee model to generate revenue and sustain marketplace operations. Sellers pay a 10% commission on every completed order, which is automatically deducted before payout. In addition, a small transaction fee of 2–3% is applied to cover payment processing costs. Shoplite does not charge a flat monthly subscription fee, allowing sellers to scale without fixed overhead.

Payouts are issued weekly, provided the seller’s account balance meets the minimum threshold of $100. If the balance is below this amount, it rolls over to the following week. Sellers can view detailed payout statements in the Seller Dashboard, including gross sales, commissions, transaction fees, and net payout amounts. Refunds and chargebacks are automatically deducted from future payouts, ensuring accurate accounting.

Optional paid services such as advertising, product boosts, and premium analytics incur additional fees. These services are billed separately and itemized clearly in seller statements. Shoplite regularly reviews its fee structure to remain competitive and transparent. Sellers are notified in advance of any changes to commissions or service fees. By offering a fair commission model without hidden costs, Shoplite maintains trust with merchants while sustaining the platform’s long-term operations."""
    },
    {
        "id": "doc10",
        "title": "Shoplite Customer Support Procedures",
        "content": """Shoplite provides multi-channel customer support to address user needs. Customers can reach support via live chat, email, or phone. Support hours are Monday through Friday, 9 AM to 5 PM local time. Live chat is recommended for urgent issues, while email is best suited for complex inquiries requiring attachments or detailed documentation. Response times are typically within 24–48 hours for emails, while chat and phone inquiries are handled in real time during business hours.

Support requests are tracked through a ticketing system that assigns each case a unique ID. Customers can monitor ticket progress from their account dashboard. For efficiency, Shoplite provides an extensive FAQ and help center that covers common topics such as account recovery, order tracking, and payment issues. Users are encouraged to consult these resources before escalating to live support.

Complex cases, such as fraud investigations or repeated delivery failures, are escalated to senior support teams. Customers are notified if their case requires additional verification or extended processing time. Shoplite also uses AI-driven chatbots to handle routine inquiries instantly, reducing wait times for human agents. By combining self-service resources, real-time communication channels, and structured escalation procedures, Shoplite ensures customers receive timely and effective assistance."""
    },
    {
        "id": "doc11",
        "title": "Shoplite Mobile App Features",
        "content": """The Shoplite mobile app extends the platform’s core functionality while adding mobile-exclusive features to improve the shopping experience. Users can log in using biometric authentication such as fingerprint or facial recognition for added convenience and security. The app supports barcode scanning, enabling users to quickly search for products by scanning items in-store or through advertisements. This feature bridges online and offline shopping experiences.

Push notifications play a central role in keeping users engaged. Customers receive instant updates on order status changes, delivery confirmations, promotions, and personalized product recommendations. Unlike email notifications, push alerts ensure real-time communication, increasing the likelihood of user engagement. Shoplite also offers app-only discounts and flash sales to incentivize mobile adoption, giving users exclusive access to limited-time deals not available on the website.

The app integrates seamlessly with the user’s account, ensuring that shopping carts, wishlists, and order histories remain synchronized across devices. Offline browsing is supported, allowing customers to view previously loaded pages and saved items without internet access. Once reconnected, the app refreshes data to reflect real-time availability. Overall, Shoplite’s mobile app enhances accessibility, introduces powerful discovery tools, and rewards users with exclusive perks that are not offered elsewhere."""
    },
    {
        "id": "doc12",
        "title": "Shoplite API Documentation for Developers",
        "content": """Shoplite provides developers with a RESTful API that enables integration of third-party applications and services. The API uses OAuth 2.0 for secure authentication and authorization. Developers can use endpoints to manage product listings, synchronize inventory, retrieve order data, and process payments securely. JSON is the standard format for requests and responses, ensuring compatibility with modern development frameworks.

Rate limits are enforced to maintain system stability, allowing up to 100 requests per minute per API key. If exceeded, requests are temporarily throttled with exponential backoff. Shoplite offers comprehensive documentation with example requests, authentication flows, and error handling guidelines. Sandbox environments are available for testing, allowing developers to simulate live scenarios without impacting production systems.

Sellers benefit from API integration by connecting Shoplite to ERP or POS systems, enabling automatic synchronization of product catalogs and order statuses. Developers can also build customer-facing apps that extend Shoplite functionality, provided they comply with marketplace policies. Security is prioritized by encrypting all API traffic via TLS and logging all access for audit purposes. This API-first approach ensures flexibility for merchants and developers while maintaining platform reliability and data integrity."""
    },
    {
        "id": "doc13",
        "title": "Shoplite Security and Privacy Policies",
        "content": """Shoplite prioritizes the protection of user data through strict security and privacy standards. All traffic is secured using TLS encryption, ensuring that sensitive information is protected in transit. User passwords are hashed and salted before storage, making them resistant to brute force and dictionary attacks. Multi-factor authentication is available for users who want an added layer of protection when logging into their accounts.

The platform complies with GDPR regulations, allowing users to request data exports and account deletion at any time. When accounts are deleted, personal identifiers are anonymized while transactional data may be retained for regulatory compliance. Shoplite does not sell customer data to third parties. Instead, data is used strictly to improve services, personalize recommendations, and ensure transaction safety.

Fraud detection algorithms continuously monitor suspicious behavior, such as multiple failed login attempts or unusual purchasing patterns. Accounts flagged for suspicious activity may be temporarily locked until verified by customer support. Shoplite also performs regular penetration testing and security audits to identify vulnerabilities. By combining technical safeguards, legal compliance, and proactive monitoring, Shoplite provides customers with a safe and trustworthy environment for online shopping."""
    },
    {
        "id": "doc14",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": """Promotional codes are an important marketing tool on Shoplite. Buyers can enter a promo code at checkout to receive discounts on eligible items. Only one code may be applied per order, and codes cannot be stacked. Codes have predefined expiration dates and may be restricted to specific product categories, sellers, or minimum order amounts. Invalid or expired codes are rejected with clear error messages.

Sellers can create promotional campaigns through the Seller Dashboard. Once submitted, campaigns are reviewed by Shoplite for compliance with marketplace policies before approval. Sellers can specify discount percentages, duration, and eligibility criteria. Shoplite tracks the performance of each campaign, providing analytics on redemption rates and sales impact. Abuse prevention mechanisms prevent excessive code usage by the same account or IP address.

Customers are notified of promotional offers via email and push notifications, with app users often receiving exclusive codes. Promotions can significantly boost sales during peak shopping seasons. By balancing seller flexibility with platform-wide safeguards, Shoplite ensures that promotions remain effective, fair, and valuable for both buyers and merchants."""
    },
    {
        "id": "doc15",
        "title": "Shoplite Warranty and Claims",
        "content": """Shoplite supports product warranties to enhance customer trust. Warranty periods typically range from six to twelve months, depending on the product category and seller policy. Customers must provide proof of purchase, such as an invoice or digital receipt, to file a warranty claim. Claims are submitted through the user dashboard, where customers can upload supporting documents and describe the defect or issue.

Once a claim is filed, Shoplite coordinates with the seller to verify the warranty eligibility. If approved, remedies may include replacement, repair, or refund. Shoplite requires sellers to process claims within a reasonable timeframe, usually ten business days. If a seller fails to respond, Shoplite may intervene directly to protect the customer. Products deemed defective on arrival are eligible for immediate replacement at no additional cost.

Warranty claims are subject to certain exclusions. Damage caused by misuse, accidents, or unauthorized modifications is not covered. Consumables and items without manufacturer warranties are excluded. Customers may be asked to return defective items before receiving replacements. By enforcing clear rules and accountability, Shoplite ensures that warranties function as a safeguard for buyers while holding sellers responsible for product quality."""
    }
]

# Load the prompts

In [ ]:
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [Your response based on context]\nSources: [List document titles referenced]"
    },
    "multi_doc_synthesis_prompt": {
        "role": "You are a policy synthesizer who combines multiple Shoplite documents into one consistent answer.",
        "goal": "Merge overlapping policies/procedures; resolve conflicts explicitly and explain assumptions when needed.",
        "context_guidelines": [
            "Reference at least two relevant documents when applicable",
            "Prefer the most specific or most recent policy if conflicts arise",
            "Call out any ambiguities and state safe, user-friendly guidance"
        ],
        "response_format": "Answer: [Concise synthesis spanning multiple docs]\nSources: [Doc A; Doc B; Doc C]"
    },
    "no_context_refusal_prompt": {
        "role": "You are a careful assistant who refuses when no relevant Shoplite context is retrieved.",
        "goal": "Avoid hallucinations; guide the user to rephrase or provide details to find the correct policy.",
        "context_guidelines": [
            "If retrieval returns nothing relevant, do not answer from general knowledge",
            "Offer 1–2 concrete suggestions for how to rephrase or what details to include"
        ],
        "response_format": "Answer: I'm sorry—I couldn't find this in the provided Shoplite documentation. Please include more details (e.g., product category, region, order ID) or rephrase your question.\nSources: None"
    },
    "clarification_needed_prompt": {
        "role": "You are an intent-clarifying assistant for Shoplite questions that are ambiguous or span multiple areas.",
        "goal": "Ask focused follow-up questions to narrow scope before answering.",
        "context_guidelines": [
            "Ask no more than 1–2 concise clarification questions",
            "Offer examples of acceptable inputs (e.g., region, payment method, order status)",
            "State which policy areas you will consult after clarification"
        ],
        "response_format": "Answer: [Brief note that clarification is needed]\nFollow-up question(s): [One or two precise questions]\nPossible sources to consult next: [List candidate document titles]"
    },
    "procedural_howto_prompt": {
        "role": "You are a step-by-step guide for executing Shoplite procedures.",
        "goal": "Provide numbered, actionable steps with prerequisites, timelines, and expected outcomes.",
        "context_guidelines": [
            "Start with a one-sentence overview",
            "List steps as a numbered sequence",
            "Include time windows (e.g., 14 days, 2–3 business days) and where to track status"
        ],
        "response_format": "Overview: [One sentence]\nSteps:\n  1) [Step one]\n  2) [Step two]\n  3) [Step three]\n  ...\nExpected outcome: [What the user should see/receive]\nSources: [List document titles referenced]"
    },
    "safety_privacy_boundary_prompt": {
        "role": "You are a safety- and privacy-aware assistant for sensitive Shoplite requests.",
        "goal": "Enforce privacy/security rules; minimize collection of personal data; direct to secure channels when needed.",
        "context_guidelines": [
            "Never request full card numbers, CVV, or passwords",
            "When identity is required, ask only for minimal necessary info (e.g., last 4 digits, order ID)",
            "For suspected account compromise, instruct password reset and contacting support",
            "Cite relevant security/privacy documents"
        ],
        "response_format": "Answer: [Safety-compliant guidance]\nSafety/Privacy Note: [Short rationale of the restriction]\nSources: [Security/Privacy document titles]"
    }
}

# Load the LLM

In [ ]:


MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"


if torch.cuda.is_available():
    compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
else:
    compute_dtype = torch.float32

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print(f"Loading {MODEL_NAME} in 4-bit (compute_dtype={compute_dtype})…")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=quant_config,
        trust_remote_code=True,
    )
    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto",
        max_new_tokens=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
    )
    print("Loaded Qwen2.5-7B-Instruct (4-bit).")
except Exception as e:
    raise RuntimeError(
        f"Failed to load {MODEL_NAME} in 4-bit. "
        f"Please free VRAM, switch runtime to GPU, or choose a smaller *Qwen* model. Error: {e}"
    )

Loading Qwen/Qwen2.5-7B-Instruct in 4-bit (compute_dtype=torch.bfloat16)…


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Loaded Qwen2.5-7B-Instruct (4-bit).


# RAG pipeline implementation

In [ ]:
EMBEDDER_NAME = "multi-qa-MiniLM-L6-cos-v1"
embedder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
# Extract docs + titles from knowledge base
docs = [d["content"] for d in KNOWLEDGE_BASE]
titles = [d["title"] for d in KNOWLEDGE_BASE]

# Create embeddings
embeddings = embedder.encode(docs, convert_to_numpy=True, normalize_embeddings=True)

# Create FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"🔹 Loaded {len(docs)} documents into FAISS index (dim={dim}) using multi-qa-MiniLM-L6-cos-v1")


def retrieve_docs(query: str, top_k: int = 3, min_score: float = 0.45):
    query_emb = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(query_emb, top_k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        if score < min_score:
            continue
        results.append({
            "rank": len(results)+1,
            "id": KNOWLEDGE_BASE[idx]["id"],
            "title": KNOWLEDGE_BASE[idx]["title"],
            "content": KNOWLEDGE_BASE[idx]["content"],
            "score": float(score),
        })
    return results


def build_prompt(prompt_key: str, query: str, retrieved):
    cfg = PROMPTS.get(prompt_key, PROMPTS["base_retrieval_prompt"])
    context_block = "\n\n".join([f"[{r['title']}]\n{r['content']}" for r in retrieved])
    role = cfg["role"]
    goal = cfg["goal"]
    guidelines = "\n".join([f"- {g}" for g in cfg["context_guidelines"]])
    rf = cfg["response_format"]

    return (
        f"{role}\n\n"
        f"Goal:\n{goal}\n\n"
        f"Context guidelines:\n{guidelines}\n\n"
        f"Retrieved document snippets:\n{context_block}\n\n"
        f"User question: {query}\n\n"
        f"{rf}"
    )

def generate_response(query: str, prompt_key: str = "base_retrieval_prompt", top_k: int = 3, max_new_tokens: int = 150):
    retrieved = retrieve_docs(query, top_k=top_k)

    if not retrieved and prompt_key != "no_context_refusal_prompt":
        prompt_key = "no_context_refusal_prompt"
    prompt = build_prompt(prompt_key, query, retrieved)

    raw = generator(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"]

    answer = raw.split("Answer:", 1)[-1].strip() if "Answer:" in raw else raw.strip()

    return {
        "model": MODEL_NAME,
        "prompt_used": prompt_key,
        "response": answer,
        "sources": [r["title"] for r in retrieved]
    }



🔹 Loaded 15 documents into FAISS index (dim=384) using multi-qa-MiniLM-L6-cos-v1


# Flask API setup

In [ ]:

app = Flask(__name__)

@app.route("/health", methods=["GET"])
def health():
    try:
        return jsonify({
            "status": "ok",
            "model": MODEL_NAME,
            "embedder": EMBEDDER_NAME,
            "docs_loaded": len(KNOWLEDGE_BASE)
        })
    except Exception as e:
        return jsonify({"status": "error", "error": str(e)}), 500


@app.route("/ping", methods=["POST"])
def ping():
    data = request.get_json(force=True) if request.data else {}
    q = (data or {}).get("query", "Hello")
    return jsonify({
        "status": "ok",
        "message": f"Pong! You said: {q}",
        "model": MODEL_NAME
    })


@app.route("/chat", methods=["POST"])
def chat():
    try:
        data = request.get_json(force=True)
        query = data.get("query", "").strip()
        prompt_key = data.get("prompt_key", "base_retrieval_prompt")
        top_k = int(data.get("top_k", 3))

        if not query:
            return jsonify({"error": "Missing 'query'"}), 400

        out = generate_response(query=query, prompt_key=prompt_key, top_k=top_k)

        # Only essentials
        return jsonify({
            "answer": out["response"],
            "sources": out["sources"]
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# ngrok token input and tunnel creation

In [ ]:
ngrok_token = input("Enter your ngrok token: ").strip()
if not ngrok_token:
    raise ValueError("ngrok token is required. Please re-run and enter your token.")

ngrok.set_auth_token(ngrok_token)


def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

server_thread = threading.Thread(target=run_app, daemon=True)
server_thread.start()

time.sleep(1)
public_url = ngrok.connect(5000, "http")
print("Your public URL:", public_url.public_url)

Enter your ngrok token: 33Nk84qdKUUGICTnGWPnB1Ja4i3_36p2uF8kBQrJ6pJfEvxgY
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Your public URL: https://wacky-rosita-laconical.ngrok-free.dev


# Testing and validation

In [ ]:
import requests
BASE = public_url.public_url

print("Health ->", requests.get(f"{BASE}/health").json())
print("Ping ->", requests.post(f"{BASE}/ping", json={"query": "say hello"}).json())

chat = requests.post(f"{BASE}/chat", json={
    "query": "What payment methods are supported on Shoplite?",
    "prompt_key": "base_retrieval_prompt",
    "top_k": 3
}).json()

print("\n--- Chat ---")
print("Answer:", chat.get("answer"))
print("Sources:", ", ".join(chat.get("sources", [])))


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 15:09:37] "GET /health HTTP/1.1" 200 -


Health -> {'docs_loaded': 15, 'embedder': 'multi-qa-MiniLM-L6-cos-v1', 'model': 'Qwen/Qwen2.5-7B-Instruct', 'status': 'ok'}


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 15:09:38] "POST /ping HTTP/1.1" 200 -


Ping -> {'message': 'Pong! You said: say hello', 'model': 'Qwen/Qwen2.5-7B-Instruct', 'status': 'ok'}


INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 15:10:02] "POST /chat HTTP/1.1" 200 -



--- Chat ---
Answer: [Your response based on context]
Sources: [List document titles referenced] According to the "Shoplite Payment Methods and Security" document, Shoplite supports multiple payment methods to accommodate user preferences. Customers can pay using Visa, Mastercard, and major debit cards. In certain regions, Cash on Delivery (COD) is also available. COD allows users to pay at the time of delivery, but refunds for COD orders are always issued via bank transfer, taking 7–10 business days depending on bank processing times. Digital wallets such as Apple Pay or PayPal are not currently supported but may be added in future updates. All online transactions are encrypted and processed through PCI-compliant payment gateways, ensuring secure transactions. Payment receipts are emailed immediately after a transaction is confirmed, and customers can download invoices from their order history page.
Sources: Shoplite Payment Methods and Security, Shoplite Commission and Fee Structure